# Notebook for running 1D-TDSE C code from Python

This notebook shows a way to generate an input file for the multiscale model consisting of:
* The pulse propagation
* TDSE
* Hankel transform

Here we show the standard operation with a pre-defined gas.$^\dagger$ The density profile is customised in this notebook.

$^\dagger$ Therefore, all the material constants are sourced from default values within the codes. 

## Load libraries

In [ ]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
import HHG
from IPython.display import display, Markdown


%matplotlib inline
# import mpld3
# mpld3.enable_notebook()

The file where the input parameters are written:

In [ ]:
h5file = os.path.join('/mnt','d','git','MMA-interactive','results.h5')

## Physical parameters

The philosophy of our input uses a "reference Gaussian beam". This means that our reference is the Gaussian beam with known parameters (focus, waist, focus intensity) propagating in vacuum. Then we add a medium in the path of the beam in our experiment. Finally, we specify the parameters of ht XUV camera.

### Laser parameters

In [ ]:
reference_Gaussian_focus = 2.5e-3 # [m]
reference_Gaussian_waist = 110e-6 # [m]
reference_Gaussian_focus_intensity = 3.6e18 # [W/m2]
laser_wavelength = 800e-9 # [m]
laser_pulse_duration = 15e-15 # [s] (defined via 1/e in the electric field amplitude)

### Medium parameters

In [ ]:
# gas specifiers
gas = 'Ar'
medium_length = 5.0e-3 # [m]
ionisation_model = 'PPT'
XUV_dispersion_tables = 'NIST'
XUV_absorption_tables = 'Henke'

# density profile (Here you can define whatever density profile you want by the function `pressure_profile(z)`.)
peak_pressure = 50e-3
relative_jet_size = 1./4.
def pressure_profile(z):
    return peak_pressure*np.exp(-((z-0.5*medium_length)/(relative_jet_size*medium_length))**2)

dz_pressure = 1e-5
zgrid_pressure = np.ogrid[0:(medium_length+dz_pressure):dz_pressure]


average_pressure = integrate.simpson(pressure_profile(zgrid_pressure),zgrid_pressure)/medium_length
pressure_modulation = pressure_profile(zgrid_pressure)/average_pressure


In [ ]:
### Code to generate the figure and the following text:
display(Markdown(rf"""### Pressure profile"""))

fig = plt.figure()
plt.plot(1e3*zgrid_pressure,1e3*average_pressure*pressure_modulation)
plt.xlabel('$z$ [mm]'); plt.ylabel('$p$ [mbar]')
# plt.title('pressure profile in the gas')
plt.show()

display(Markdown(rf"""
The average pressure is $\bar{{p}}={1e3*average_pressure:.2f}~{{\mathrm{{mbar}}}}$ (this value will be used as the reference for the CUPRAD propagation).
"""))

### XUV camera

In [ ]:
XUV_camera_distance         = 1. # [m] (from the entry of the cell)
XUV_camera_harmonic_range   = np.asarray([14., 60.]) # [harmonic order]
XUV_camera_radial_range     = 0.007 # [m]

## Numerical parameters

Here we define the numerical parameters. This release of the code leaves the responsibility of choosing proper parameters to users, except the implementatation of adaptive steps in $z$.

### CUPRAD (pulse propagation)

In [ ]:
number_of_points_in_r      = 1024
number_of_points_in_t      = 2048

operators_t                =	2
first_delta_z     =	0.01 # [mm]
phase_threshold_for_decreasing_delta_z = 0.002	# [rad]

length_of_window_for_r_normalized_to_beamwaist = 4.   # [-]
length_of_window_for_t_normalized_to_pulse_duration = 12. # [-]

number_of_absorber_points_in_time = 16  # [-]

physical_output_distance_for_plasma_and_Efield = 0.00001   # [m]

output_distance_in_z_steps_for_fluence_and_power   = 100  # [-]

radius_for_diagnostics = 0.1 # [cm]

run_time_in_hours = 5.0 # [h] 

In [ ]:
## Code to generate the following text ##
zR = (np.pi*reference_Gaussian_waist**2)/laser_wavelength
dr_CUPRAD = length_of_window_for_r_normalized_to_beamwaist * reference_Gaussian_waist*np.sqrt(1+(reference_Gaussian_focus/zR)**2)/number_of_points_in_r
display(Markdown(rf"""### Properties of the chosen discretisation
* The chosen discretisation in time gives ~ {
            number_of_points_in_t/(
            laser_pulse_duration*length_of_window_for_t_normalized_to_pulse_duration/mn.ConvertPhoton(laser_wavelength,'lambdaSI','T0SI')
            )
    :.0f}
points per one laser period.
* The stepsize in the radial discretisation is ~ ${
      1e6*dr_CUPRAD
      :.2f}
~\mu {{\mathrm{{m}}}}$.
* The size of the radial computational box is ~ ${
      1e6*length_of_window_for_r_normalized_to_beamwaist * reference_Gaussian_waist
      :.2f}
~\mu {{\mathrm{{m}}}}$. The maximal radius of the reference Gaussian beam is ~ ${
      1e6*np.max([
            reference_Gaussian_waist*np.sqrt(1+((medium_length-reference_Gaussian_focus)/zR)**2),
            reference_Gaussian_waist*np.sqrt(1+(reference_Gaussian_focus/zR)**2)
            ])
      :.2f}
~\mu {{\mathrm{{m}}}}$.$^\dagger$ 

$^\dagger$ This is given at the $z$-edges of the copmutational box.
"""))

### TDSE

In [ ]:
# Macroscopic: to slect the grid based on the CUPRAD grid
kz_step = 1
kr_step_CTDSE = 2
Nr_max = 400

# Microscopic part
dt_TDSE = 0.25 # [a.u.]
dx      = 0.4  # [a.u.]
Nx_max  = 16000 # (spans from -dx*Nx_max to dx*Nx_max)
x_int   = 2.0  # [a.u.]

CV_criterion_of_ground_state = 1e-12 # [-]

# Outputs
# choose from: 'electric field', 'electric field (Fourier),
#              '<dj/dt>', '<dj/dt> (Fourier)'
#              'ground-state population (projected)', 'ground-state population (integrated)'
#              '<x>'
list_of_CTDSE_outputs = ['electric field', '<dj/dt> (Fourier)',
                         'ground-state population (projected)', '<x>']

In [ ]:
## Code to generate the following text ##

onax_entry_intensity = (reference_Gaussian_focus_intensity/units.INTENSITYau)/np.sqrt(1+(reference_Gaussian_focus/zR)**2)
onax_entry_ponderomotive_potential = onax_entry_intensity/(4.*mn.ConvertPhoton(laser_wavelength,'lambdaSI','omegaau'))
max_Ek_direct = 5.*onax_entry_ponderomotive_potential
max_electron_velocity_direct = np.sqrt(2.*max_Ek_direct)
max_Ek_rescattered = 10.*onax_entry_ponderomotive_potential
max_electron_velocity_rescattered = np.sqrt(2.*max_Ek_rescattered)
t_box = (length_of_window_for_t_normalized_to_pulse_duration*laser_pulse_duration)/units.TIMEau

display(Markdown(rf"""### Physical consequences of the chosen numerical parameters
* The step-size in $z$ is derived from the CUPRAD's adaptive steps, the stride for CTDSE is {kz_step}. (Generally, we do not recommend to use stride > 1.$^\dagger$)
* The macroscopic radial discretisation for TDSE is ${
      1e6*kr_step_CTDSE*dr_CUPRAD  
      :.2f}
~\mu {{\mathrm{{m}}}}$; the macroscopic radial boxsize is $r_{{\text{{max}}}}={
    1e6*Nr_max*dr_CUPRAD
    :.2f}
~\mu {{\mathrm{{m}}}}$.
* The outputs stored from CTDSE runs are: {', '.join(['***'+foo+'***' for foo in list_of_CTDSE_outputs])}.
* Possible ouputs *ground-state population (projected)* and *ground-state population (integrated)* refer to various approaches to ionisation. See
[link 1](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.106.053115) or [link 2](https://theses.hal.science/tel-04192431v1/document)$^{{\dagger\dagger}}$ (Chapter 3) for details.
* The microscopic computatinal box for 1D-TDSE is $x_{{\text{{max}}}} = {
    dx*Nx_max
    :.0f}~{{\mathrm{{a.u.}}}}~({
    1e9*dx*Nx_max * units.LENGTHau    
    :.2f}~{{\mathrm{{nm}}}})$.
* The maximal energy in the spectrum according to the chosen discretisation is $E_{{\text{{max}}}} = {
    mn.ConvertPhoton((2.*np.pi/dt_TDSE),'omegaau','eV')
    :.2f}~{{\mathrm{{eV}}}}$ ($H_{{\text{{max}}}} \sim {
    (2.*np.pi/dt_TDSE)/mn.ConvertPhoton(laser_wavelength,'lambdaSI','omegaau')
    :.0f}$).
* Theoretical maximal distances of a classical electron ejected at the peak of the pulse reached at,
respectively, the end of the compuational box and the trailing edge of the pulse ($1/\mathrm{{e}}^2$ of the intensity).
    * [Direct electrons with $E_{{\text{{kin}}}} \sim 5U_p$](https://doi.org/10.1103/PhysRevA.106.053115) $s_{{\text{{max}}}} = {
    max_electron_velocity_direct*0.5*t_box
    :.2f}~{{\mathrm{{a.u.}}}}~({
    1e9*max_electron_velocity_direct*0.5*t_box*units.LENGTHau    
    :.2f}~{{\mathrm{{nm}}}})$.
    * [Rescatterred electrons $E_{{\text{{kin}}}} \sim 10U_p$](https://doi.org/10.1038/nphys914) $s_{{\text{{max}}}} = {
    max_electron_velocity_rescattered*0.5*t_box
    :.2f}~{{\mathrm{{a.u.}}}}~({
    1e9*max_electron_velocity_rescattered*0.5*t_box*units.LENGTHau    
    :.2f}~{{\mathrm{{nm}}}})$, $s_{{\text{{max,2}}}} = {
    max_electron_velocity_rescattered*0.5*(laser_pulse_duration/units.TIMEau)
    :.2f}~{{\mathrm{{a.u.}}}}~({
    1e9*max_electron_velocity_rescattered*0.5*(laser_pulse_duration/units.TIMEau)*units.LENGTHau    
    :.2f}~{{\mathrm{{nm}}}})$.
* The cut-off for the peak on-axis entry intensity is $H_{{\text{{cut-off}}}} = {
   HHG.ComputeCutoff_gas(onax_entry_intensity,mn.ConvertPhoton(laser_wavelength,'lambdaSI','omegaau'),gas=gas)[1]
    :.2f}$ (given by $I_P + 3.17 U_p$).


$^\dagger$ The step-size in $z$ is usually adapted such that the variations of the phase are affecting XUV when using a coarser grid. \
$^{{\dagger\dagger}}$ *ground-state population (projected)* corresponds to Eq. (3.20) and *ground-state population (projected)* to Eq. (3.1) of [link 2](https://theses.hal.science/tel-04192431v1/document).
"""))

### Hankel

In [ ]:
store_cummulative_field = True
kr_step_Hankel                   = 1
ko_step                          = 2
Nr_max_Hankel_integration        = 235
XUV_camera_number_of_r_points    = 600

Nthreads = 24

In [ ]:
## Code to generate the following text ##
dr_Hankel = kr_step_CTDSE*kr_step_Hankel*dr_CUPRAD


first_diffraction_maximum_cutoff = laser_wavelength/\
        (dr_Hankel*HHG.ComputeCutoff_gas(onax_entry_intensity,mn.ConvertPhoton(laser_wavelength,'lambdaSI','omegaau'),gas=gas)[1])

display(Markdown(rf"""### The role of parameters of the camera and the integration
* The macroscopic radial discretisation for Hankel integral is ${
      1e6*dr_Hankel   
      :.2f}
~\mu {{\mathrm{{m}}}}$; the macroscopic radial boxsize is $r_{{\text{{max, integration}}}}={
    1e6*Nr_max_Hankel_integration*dr_Hankel 
    :.2f}
~\mu {{\mathrm{{m}}}}$.
* The camera size, $r_{{\text{{max, camera}}}}={
    1e3*XUV_camera_radial_range/XUV_camera_distance
    :.2f}
~{{\mathrm{{mm}}}}$, gives the maximal divergence recorded by the XUV is $\theta_{{\text{{max, camera}}}}={
    1e3*np.arctan(XUV_camera_radial_range/XUV_camera_distance ) 
    :.2f}
~{{\mathrm{{mrad}}}}$. (See the initial section with the physical parameters.)
* The diffraction limit for the maximal expected cut-off provided by the discretisation in the integral is  $r_{{\text{{max, camera, cut-off}}}}={
    1e3*first_diffraction_maximum_cutoff
    :.2f}
~{{\mathrm{{mm}}}}$ (corresponding divergence $\theta_{{\text{{max, camera, cut-off}}}}={
    1e3*np.arctan(first_diffraction_maximum_cutoff/XUV_camera_distance ) 
    :.2f}
~{{\mathrm{{mrad}}}}$)
"""))

# print(laser_wavelength/(50.*dr_Hankel))

## Prepare the input file

Here we create the HDF5 file containing all the input parameters. First, we provide several dictionaries (for different modules) to translate the local variables usedin this jupyter notebook to the nomenclature used in the code. Second, we crreate the archive. Density modulation is treated in the second part without a dictionary.

In [ ]:
# Code to create the input hdf5-file
## First, we prepare dictionaries between hdf5-inputs and this jupyter notebook

global_input_names_to_jupyter_variables = {
    'gas_preset'                                : (np.string_(gas),                       '[-]'   ),
    'medium_pressure_in_bar'                    : (average_pressure,                      '[bar]' )
}


CUPRAD_names_to_jupyter_variables = {
    # laser parameters
    'laser_wavelength'                          : (1e2*laser_wavelength,                  '[cm]'  ),
    'laser_pulse_duration_in_1_e_Efield'        : (1e15*laser_pulse_duration,             '[fs]' ),
    'laser_focus_intensity_Gaussian'            : (reference_Gaussian_focus_intensity,    '[s]'  ),
    'laser_focus_beamwaist_Gaussian'            : (reference_Gaussian_waist,              '[m]'  ),
    'laser_focus_position_Gaussian'             : (reference_Gaussian_focus,              '[m]'  ),

    # medium parameters
    'medium_physical_distance_of_propagation'   : (medium_length,                         '[m]'   ),

    # ionisation
    'ionization_model'                          : (np.string_(ionisation_model),          '[s]'  ),

    # numerics
    'numerics_number_of_points_in_r'            : (number_of_points_in_r,                 '[-]'  ),
    'numerics_number_of_points_in_t'            : (number_of_points_in_t,                 '[-]'  ),
    'numerics_operators_t_t-1'                  : (operators_t,                           '[-]'  ),
    'numerics_physical_first_stepwidth'         : (first_delta_z,                         '[mm]' ),
    'numerics_phase_threshold_for_decreasing_delta_z' : 
        (phase_threshold_for_decreasing_delta_z,                '[rad]' ),
    'numerics_length_of_window_for_r_normalized_to_beamwaist':
        (length_of_window_for_r_normalized_to_beamwaist,        '[-]'   ),
    'numerics_length_of_window_for_t_normalized_to_pulse_duration' :
        (length_of_window_for_t_normalized_to_pulse_duration,   '[-]'   ),
    'numerics_number_of_absorber_points_in_time':
        (number_of_absorber_points_in_time ,                    '[-]'   ),
    'numerics_physical_output_distance_for_plasma_and_Efield' :
        (physical_output_distance_for_plasma_and_Efield,        '[m]'   ),
    'numerics_output_distance_in_z-steps_for_fluence_and_power' :
        (output_distance_in_z_steps_for_fluence_and_power,      '[-]'   ),
    'numerics_radius_for_diagnostics'           : (radius_for_diagnostics,                '[cm]' ),
    'numerics_run_time_in_hours'                : (run_time_in_hours,                     '[s]'  )
}


CTDSE_names_to_jupyter_variables = {
    # Physics
    'x_int'                                     : (x_int,                                 '[a.u.]' ),

    # Macro grid
    'Nr_max'                                    : (Nr_max,                                '[-]'    ),
    'kr_step'                                   : (kr_step_CTDSE,                               '[-]'    ),
    'kz_step'                                   : (kz_step,                               '[-]'    ),  

    # Microscopic numerics
    'dx'                                        : (dx,                                    '[a.u.]' ),
    'Nx_max'                                    : (Nx_max,                                '[a.u.]' ),
    'dt'                                        : (dt_TDSE,                               '[a.u.]' ),

    'CV_criterion_of_GS'                        : (CV_criterion_of_ground_state,          '[a.u.]')
}


CTDSE_outputs_to_jupyter_names = {
    'print_Efield'                : 'electric field',
    'print_F_Efield'              : 'electric field (Fourier)',
    'print_Source_Term'           : '<dj/dt>',
    'print_F_Source_Term'         : '<dj/dt> (Fourier)',
    'print_GS_population'         : 'ground-state population (projected)',
    'print_integrated_population' : 'ground-state population (integrated)',
    'print_x_expectation_value'   : '<x>'}

Hankel_names_to_jupyter_variables = {
    'distance_FF'                               : (XUV_camera_distance,                   '[m]'  ),
    'rmax_FF'                                   : (XUV_camera_radial_range,               '[m]'  ),
    'Nr_FF'                                     : (XUV_camera_number_of_r_points,         '[-]'  ),

    'XUV_table_type_dispersion'                 : (np.string_(XUV_dispersion_tables),     '[-]'  ),
    'XUV_table_type_absorption'                 : (np.string_(XUV_absorption_tables),     '[-]'  ),

    'kr_step'                                   : (kr_step_Hankel,                               '[-]'  ),
    'ko_step'                                   : (ko_step,                               '[-]'  ),
    'Nr_max'                                    : (Nr_max_Hankel_integration,             '[-]'  ),
    'Harmonic_range'                            : (XUV_camera_harmonic_range,      '[harmonic order]'),

    'store_cummulative_result'                  : (int(store_cummulative_field),          '[-]'  ),   
    'Nthreads'                                  : (Nthreads,                              '[-]'  ) 
}


## Create the hdf5-archive
with h5py.File(h5file,'w') as f:

    # global inputs
    global_inputs = f.create_group(MMA.paths['global_inputs'])
    for dset_name, (value, unit) in global_input_names_to_jupyter_variables.items():
        mn.adddataset(global_inputs,dset_name,value,unit)
    
    density_modulation = global_inputs.create_group('density_mod')
    mn.adddataset(density_modulation ,'table',pressure_modulation,'[-]')
    mn.adddataset(density_modulation ,'zgrid',zgrid_pressure,'[m]')

    
    # CUPRAD
    CUPRAD_inps = f.create_group(MMA.paths['CUPRAD_inputs'])
    for dset_name, (value, unit) in CUPRAD_names_to_jupyter_variables.items():
        mn.adddataset(CUPRAD_inps,dset_name,value,unit)


    # CTDSE
    # regular inputs
    CTDSE_inps = f.create_group(MMA.paths['CTDSE_inputs'])
    for dset_name, (value, unit) in CTDSE_names_to_jupyter_variables.items():
        mn.adddataset(CTDSE_inps,dset_name,value,unit)

    # CTDSE prints
    mn.adddataset(CTDSE_inps,'print_GS',1.,'[-]')                        
    for CTDSE_name, jupyter_name in CTDSE_outputs_to_jupyter_names.items():
        mn.adddataset(CTDSE_inps,CTDSE_name,int(jupyter_name in list_of_CTDSE_outputs),'[-]')

    # obsolete inputs: to be removed before the release
    for dset_name in ('print_F_Efield_M2','print_F_Source_Term_M2','InterpByDTorNT'):
        mn.adddataset(CTDSE_inps,dset_name,0,'[-]')
    mn.adddataset(CTDSE_inps,'Ntinterp',1,'[-]')


    # Hankel
    Hankel_inps = f.create_group(MMA.paths['Hankel_inputs'])
    for dset_name, (value, unit) in Hankel_names_to_jupyter_variables.items():
        mn.adddataset(Hankel_inps,dset_name,value,unit)